In [ ]:
# !pip install transformers

In [ ]:
# !pip install tf-keras

In [ ]:
from transformers import AutoTokenizer,TFAutoModel
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-mini")
model = TFAutoModel.from_pretrained("prajjwal1/bert-mini",from_pt=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'bert.embeddings.position_ids', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [ ]:
import pandas as pd
df = pd.read_csv('HateSpeechDatasetBalanced.csv')
df.head()

,Content,Label
0,denial of normal the con be asked to comment o...,1
1,just by being able to tweet this insufferable ...,1
2,that is retarded you too cute to be single tha...,1
3,thought of a real badass mongol style declarat...,1
4,afro american basho,1


In [ ]:
df.drop_duplicates(subset=['Content'],inplace=True)
df.dropna(subset=['Content'],inplace=True)
df = df[df['Content'].str.len()>1]
df = df.sample(frac=0.6, random_state=1)

In [ ]:
len(df)

420023

In [ ]:
encoded_inputs = tokenizer(
    df['Content'].tolist(),
    padding = True,
    truncation=True,
    return_tensors='tf',
    max_length=100
)

In [ ]:
batch_size = 20
outputs = []

for i in range(0, len(encoded_inputs['input_ids']), batch_size):
    batch_input_ids = encoded_inputs['input_ids'][i:i+batch_size]
    batch_attention_mask = encoded_inputs['attention_mask'][i:i+batch_size]

    # 将输入放入模型中并获取输出
    batch_output = model({'input_ids': batch_input_ids, 'attention_mask': batch_attention_mask})
    outputs.append(batch_output)

In [ ]:
import tensorflow as tf

In [ ]:
final_output = tf.concat([output['last_hidden_state'] for output in outputs], axis=0)

In [ ]:
final_output

<tf.Tensor: shape=(420023, 100, 256), dtype=float32, numpy=
array([[[-0.57189536, -0.31680027,  0.9213375 , ..., -0.6004787 ,
         -0.37954748, -0.777109  ],
        [-1.7749273 ,  0.19596325, -0.01406603, ..., -0.45159274,
          0.5276122 , -0.8990034 ],
        [-0.40558645,  0.39922947,  1.3492923 , ..., -0.50554985,
          0.0524192 , -0.27326846],
        ...,
        [-0.5080751 ,  0.2841594 ,  0.8123927 , ..., -0.4148425 ,
          0.05730512, -0.6146263 ],
        [-0.01994731, -0.38263667,  0.45493418, ..., -1.037519  ,
          0.3458075 , -0.81005913],
        [ 0.5096139 , -0.7808671 ,  0.67695636, ...,  0.15591241,
          0.92076904,  0.18061565]],

       [[-0.89056194, -0.3459822 ,  0.9646132 , ..., -0.3464976 ,
         -1.3829161 , -1.1503345 ],
        [-1.5024074 , -1.0287977 ,  0.695825  , ..., -0.83375293,
         -1.2247434 , -1.4466686 ],
        [-1.7359278 ,  0.7637445 ,  0.17577446, ..., -0.503555  ,
          0.03031102, -1.178607  ],
       

In [ ]:
y = df['Label']

In [ ]:
len(y)

420023

In [ ]:
final_output = final_output.numpy()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(final_output,y,random_state=1)

In [ ]:
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,random_state=1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from tensorflow.keras import regularizers

In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(100, return_sequences=True, input_shape=(100,256)))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(100))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
lstm_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 100)          142800    
                                                                 
 dropout (Dropout)           (None, 100, 100)          0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 223301 (872.27 KB)
Trainable params: 223301 (872.27 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
lstm_model.fit(x_train, y_train, epochs=20, validation_data=(x_val,y_val),verbose=1)

Epoch 1/20
7384/7384 [==============================] - 977s 132ms/step - loss: 0.4360 - accuracy: 0.7937 - val_loss: 0.4228 - val_accuracy: 0.8025
Epoch 2/20
7384/7384 [==============================] - 967s 131ms/step - loss: 0.3765 - accuracy: 0.8272 - val_loss: 0.3805 - val_accuracy: 0.8222
Epoch 3/20
7384/7384 [==============================] - 967s 131ms/step - loss: 0.3476 - accuracy: 0.8426 - val_loss: 0.3655 - val_accuracy: 0.8351
Epoch 4/20
7384/7384 [==============================] - 968s 131ms/step - loss: 0.3253 - accuracy: 0.8551 - val_loss: 0.3685 - val_accuracy: 0.8333
Epoch 5/20
7384/7384 [==============================] - 967s 131ms/step - loss: 0.3063 - accuracy: 0.8641 - val_loss: 0.3535 - val_accuracy: 0.8399
Epoch 6/20
7384/7384 [==============================] - 973s 132ms/step - loss: 0.2880 - accuracy: 0.8737 - val_loss: 0.3659 - val_accuracy: 0.8416
Epoch 7/20
7384/7384 [==============================] - 970s 131ms/step - loss: 0.2713 - accuracy: 0.8815 - val_

In [ ]:
lstm_model.evaluate(x_train,y_train)

In [ ]:
lstm_model.evaluate(x_val,y_val)

In [ ]:
lstm_model.evaluate(x_test,y_test)